In [26]:
import torch
import torch.nn as nn
import data.datasets as dataSource
import torch.utils.data as tdata
import numpy as np
import scipy.constants as _scipy_constants

import cvxpy as cp


In [6]:
dataset_train, dataset_val = dataSource.Dipole_H_train, dataSource.Dipole_H_val
batch_size = 64
important_cols = ["B_design", "fieldTolerance", "maxCurrentDensity", "aper_x", "aper_y", "B_real"]
important_cols_order = sorted(important_cols,key=lambda c: dataset_train.input_columns.index(c))

print(important_cols)
print(important_cols_order)

input_mask = [True if col in important_cols else False for col in dataset_train.input_columns]
train_loader, val_loader = tdata.DataLoader(dataset_train,batch_size), tdata.DataLoader(dataset_val,batch_size)

target_index_dict = {col:i for i,col in enumerate(dataset_train.target_columns)}
target_cols = ["B0","gfr_x_1e-4","gfr_y_1e-4"]
output_mask = [True if col in target_cols else False for col in dataset_train.target_columns]

['B_design', 'fieldTolerance', 'maxCurrentDensity', 'aper_x', 'aper_y', 'B_real']
['maxCurrentDensity', 'fieldTolerance', 'aper_x', 'aper_y', 'B_design', 'B_real']


In [125]:
dataset_train.dataframe["B_design"].mean()

1.2925552815282366

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
oid = {key : important_cols.index(key) for key in important_cols_order}

def get_dipole_continuous(x: torch.Tensor):
    # x.shape = (b,6) with batchDim b and 6 "actual" inputs 'important cols'
    aper_x = x[:,oid["aper_x"]]
    aper_y = x[:,oid["aper_y"]]
    maxCurrentDensity = x[:,oid["maxCurrentDensity"]]
    fieldTolerance = x[:,oid["fieldTolerance"]]
    B_design = x[:,oid["B_design"]]
    B_real = x[:,oid["B_real"]]

    gfr_x = aper_x - 0.015
    gfr_y = aper_y - 0.015
    aper_x_poleoverhang = aper_x * 0.5 * (-0.36 * torch.log(fieldTolerance) - 0.90)
    aper_x_tapering = aper_x_poleoverhang * 0.1
    aper_y_distFromCoil = aper_y * 0.1
    B_max = B_design * (1 + 20 * 0.01)
    totalCurrent = B_real * aper_y / (2 * _scipy_constants.mu_0)
    totalCurrentMax = B_max * aper_y / (2 * _scipy_constants.mu_0)

    # windings and related properties dont work exactly, as the ceil function is not continuous
    # nevertheless, the continuous approximations could still be useful for the network
    #windings_cont = totalCurrentMax / maxCurrent
    
    w = aper_x + 2 * aper_x_poleoverhang
    w_leg = 0*5 * (w + 2 * aper_y) * B_max / 2.15

    minCoilArea = totalCurrentMax / (maxCurrentDensity * (75 * 0.01)) / 1e6

    coil_width = torch.sqrt(minCoilArea * 2)
    coil_height = minCoilArea / coil_width

    yoke_x = w + 2 * (aper_x_tapering + w_leg + coil_width)
    yoke_y = aper_y + 2 * (aper_y_distFromCoil + w_leg + coil_height)

    stack = torch.cat([
        x,
        gfr_x.unsqueeze(1),
        gfr_y.unsqueeze(1),
        aper_x_poleoverhang.unsqueeze(1),
        aper_x_tapering.unsqueeze(1),
        aper_y_distFromCoil.unsqueeze(1),
        B_max.unsqueeze(1),
        totalCurrent.unsqueeze(1),
        totalCurrentMax.unsqueeze(1),
        w.unsqueeze(1),
        w_leg.unsqueeze(1),
        minCoilArea.unsqueeze(1),
        coil_width.unsqueeze(1),
        coil_height.unsqueeze(1),
        yoke_x.unsqueeze(1),
        yoke_y.unsqueeze(1),
        ],dim=1)
    return stack

In [82]:
class MyRegNet(nn.Module):
    def __init__(self,h_size=8):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(len(important_cols)+15, dtype=torch.double)
        self.fc1 = nn.Linear(len(important_cols)+15,h_size, dtype=torch.double)
        self.nl1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm1d(h_size, dtype=torch.double)
        #self.do1 = nn.Dropout1d(0.5)
        #self.fc2 = nn.Linear(h_size,h_size, dtype=torch.double)
        #self.nl2 = nn.LeakyReLU()
        #self.bn2 = nn.BatchNorm1d(h_size, dtype=torch.double)
        #self.do2 = nn.Dropout1d()
        self.fc3 = nn.Linear(h_size,3, dtype=torch.double)
    def forward(self,x):
        x = get_dipole_continuous(x)
        x = self.bn0(x)
        x = self.fc1(x)
        x = self.nl1(x)
        x = self.bn1(x)
        #x = self.do1(x)
        #x = self.fc2(x)
        #x = self.nl2(x)
        #x = self.bn2(x)
        #x = self.do2(x)
        x = self.fc3(x)
        return x

network = MyRegNet().to(device)

In [83]:
loss_mask_df = dataset_train.dataframe[["B0","gfr_x_1e-4","gfr_y_1e-4"]].mean()


In [84]:
n_epochs = 200
lr = 1e-4
optim = torch.optim.Adam(network.parameters(), lr)
loss_fun = nn.MSELoss()

for epoch in range(n_epochs):
    network.train()
    loss_list = []
    for x,y in train_loader:
        selected_x = x[:,input_mask].double().to(device)
        selected_y = y[:,output_mask].double().to(device)
        pred = network(selected_x)

        loss_mask = (torch.tensor(list(loss_mask_df))* torch.tensor([1.5,1,1])).repeat((x.shape[0],1)).to(device)
        loss = loss_fun(pred * loss_mask,selected_y * loss_mask)
        loss_list.append(loss.detach().item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    epoch_train_loss = torch.tensor(loss_list).mean()

    network.eval()
    loss_list = []
    for x,y in val_loader:
        selected_x = x[:,input_mask].double().to(device)
        selected_y = y[:,output_mask].double().to(device)
        pred = network(selected_x)

        loss_mask = (torch.tensor(list(loss_mask_df))* torch.tensor([1.5,1,1])).repeat((x.shape[0],1)).to(device)
        loss = loss_fun(pred * loss_mask,selected_y * loss_mask)
        loss_list.append(loss.detach().item())
    epoch_val_loss = torch.tensor(loss_list).mean()
    print(epoch,epoch_train_loss,epoch_val_loss,end="\r")


KeyboardInterrupt: 

In [85]:
torch.save(network.state_dict(), "network_design_params")

In [86]:

network = MyRegNet().to(device)
network.load_state_dict(torch.load("network_design_params"))

<All keys matched successfully>

In [116]:
A = []
b = []

def onehot(name, neg):
    out = [0] * len(important_cols_order)
    out[important_cols_order.index(name)] = -1 if neg else 1
    return out

for name in important_cols_order:
    # constraints for minima: x > a --> -x < -a
    A.append(onehot(name,True))
    b.append(-dataset_train.input_mins[name])
    
    # constraints for maxima: x < a
    A.append(onehot(name,False))
    b.append(dataset_train.input_maxs[name])

In [117]:
A_rel = []
b_rel = []

# ['maxCurrentDensity', 'fieldTolerance', 'aper_x', 'aper_y', 'B_design', 'B_real']

# aper_x > aper_y --> aper_y - aper_x < 0
A_rel.append([0, 0, -1, 1, 0, 0])
b_rel.append(0)

# aper_x < 5 * aper_y --> aper_x - 5 * aper_y < 0
A_rel.append([0, 0, 1, -5, 0, 0])
b_rel.append(0)

# 1.2 * B_design > B_real --> B_real - 1.2 * B_design < 0
A_rel.append([0, 0, 0, 0, -1.2, 1])
b_rel.append(0)


A = A + A_rel
b = b + b_rel

In [118]:
# ratio constraints 
# compute min and max ratio for every pair of inputs and ensure the ratio is in that range with two linear constraints
# e.g. min < a/b < max -->  min * b - a < 0 and a - max * b < 0
# with slack. min * (1+e) < a/b < max * (1-e) -->  (min (1+ e)) * b - a < 0 and a - (max*(1-e)) * b < 0

A_rat = []
b_rat = []
epsilon = 1e-5

for i in range(len(important_cols_order)-1):
    for j in range(i+1,len(important_cols_order)):
        quantity_1 = important_cols_order[i]
        quantity_2 = important_cols_order[j]

        # ratio = column i/ colimn j
        ratio_df = dataset_train.dataframe.assign(ratio=dataset_train.dataframe.apply(lambda r: r[quantity_1] / r[quantity_2], axis=1))

        minimum = ratio_df["ratio"].min() 
        maximum = ratio_df["ratio"].max()

        minimum *= 1+epsilon
        maximum *= 1-epsilon

        # min * j - i < 0
        min_row = [0]*6
        min_row[i] = -1
        min_row[j] = minimum
        A_rat.append(min_row)
        b_rat.append(0)

        #i - max * k < 0
        max_row = [0]*6
        max_row[i] = 1
        max_row[j] = -maximum
        A_rat.append(max_row)
        b_rat.append(0)

        print(quantity_1,quantity_2,minimum,maximum, "-->", min_row,"< 0 and", max_row,"< 0")


A = A + A_rat
b = b + b_rat

maxCurrentDensity fieldTolerance 200.00001 99999.99999 --> [-1, 200.00001, 0, 0, 0, 0] < 0 and [1, -99999.99999, 0, 0, 0, 0] < 0
maxCurrentDensity aper_x 6.666676666666667 399.99999 --> [-1, 0, 6.666676666666667, 0, 0, 0] < 0 and [1, 0, -399.99999, 0, 0, 0] < 0
maxCurrentDensity aper_y 6.666676666666667 399.99999 --> [-1, 0, 0, 6.666676666666667, 0, 0] < 0 and [1, 0, 0, -399.99999, 0, 0] < 0
maxCurrentDensity B_design 1.00001 24.99999 --> [-1, 0, 0, 0, 1.00001, 0] < 0 and [1, 0, 0, 0, -24.99999, 0] < 0
maxCurrentDensity B_real 0.8333433333333333 249.99999 --> [-1, 0, 0, 0, 0, 0.8333433333333333] < 0 and [1, 0, 0, 0, 0, -249.99999] < 0
fieldTolerance aper_x 0.0003433333333333334 0.39998999999999996 --> [0, -1, 0.0003433333333333334, 0, 0, 0] < 0 and [0, 1, -0.39998999999999996, 0, 0, 0] < 0
fieldTolerance aper_y 0.0003433333333333334 0.39998999999999996 --> [0, -1, 0, 0.0003433333333333334, 0, 0] < 0 and [0, 1, 0, -0.39998999999999996, 0, 0] < 0
fieldTolerance B_design 6e-05 0.016656666

In [119]:
def project_to_poly2(x, A_mat=A, b_vec = b):
    # Define decision variable
    p = cp.Variable(len(x))
    # Define objective function
    objective = cp.Minimize(cp.sum_squares(p - x))

    A_mat_dynamic = A_mat[:]
    b_vec_dynamic = b_vec[:]


    # Define constraints
    constraints = [np.array(A_mat) @ p <= np.array(b_vec)]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()


    # Optimal solution
    closest_point = p.value
    return closest_point

A_test = [[0,1],[1,0]]
b_test = [1,1]
project_to_poly2([0.5,1.2],A_test,b_test)

array([0.5, 1. ])

In [110]:
list(zip(A,b))

[([-1, 0, 0, 0, 0, 0], -2.0),
 ([1, 0, 0, 0, 0, 0], 10.0),
 ([0, -1, 0, 0, 0, 0], -0.0001),
 ([0, 1, 0, 0, 0, 0], 0.01),
 ([0, 0, -1, 0, 0, 0], -0.025),
 ([0, 0, 1, 0, 0, 0], 0.3),
 ([0, 0, 0, -1, 0, 0], -0.025),
 ([0, 0, 0, 1, 0, 0], 0.3),
 ([0, 0, 0, 0, -1, 0], -0.4),
 ([0, 0, 0, 0, 1, 0], 2.0),
 ([0, 0, 0, 0, 0, -1], -0.04),
 ([0, 0, 0, 0, 0, 1], 2.4),
 ([0, 0, -1, 1, 0, 0], 0),
 ([0, 0, 1, -5, 0, 0], 0),
 ([0, 0, 0, 0, -1.2, 1], 0),
 ([-1, 200.0, 0, 0, 0, 0], 0),
 ([1, -100000.0, 0, 0, 0, 0], 0),
 ([-1, 0, 6.666666666666667, 0, 0, 0], 0),
 ([1, 0, -400.0, 0, 0, 0], 0),
 ([-1, 0, 0, 6.666666666666667, 0, 0], 0),
 ([1, 0, 0, -400.0, 0, 0], 0),
 ([-1, 0, 0, 0, 1.0, 0], 0),
 ([1, 0, 0, 0, -25.0, 0], 0),
 ([-1, 0, 0, 0, 0, 0.8333333333333334], 0),
 ([1, 0, 0, 0, 0, -250.0], 0),
 ([0, -1, 0.0003333333333333334, 0, 0, 0], 0),
 ([0, 1, -0.39999999999999997, 0, 0, 0], 0),
 ([0, -1, 0, 0.0003333333333333334, 0, 0], 0),
 ([0, 1, 0, -0.39999999999999997, 0, 0], 0),
 ([0, -1, 0, 0, 5e-05, 0], 0

In [133]:
target = torch.tensor([1.8,0.14,0.07], device=device,dtype=torch.double)
loss_fun = torch.nn.MSELoss()

In [136]:
# init guess
guess = torch.randn((6),dtype=torch.double)
guess = network.bn0.running_mean[:6].clone().detach()

initial_current_density = 3
guess[0] = initial_current_density

# initially project to ensure all values make sense and the continuous transformations make sense  as well
magnet_params = torch.tensor(project_to_poly2(guess.detach().cpu().numpy())).unsqueeze(0).to(device).requires_grad_(True)

lr = 1e-1
lr_tensor = network.bn0.running_mean[:6].clone().detach() * lr
print(lr_tensor)
network.eval()

n_steps = 1000

for step in range(n_steps):
    if magnet_params.grad is not None:
        magnet_params.grad.zero_()
    for layer in network.parameters():
        if layer.grad is not None:
            layer.grad.zero_()

    pred = network(magnet_params)
    #print(pred)
    loss = loss_fun(pred,target)
    #print(loss)

    print(f"{step}, {loss.detach().cpu().numpy():.12f}",end="\r")
    loss.backward()
    with torch.no_grad():        
        magnet_params = magnet_params - lr_tensor * magnet_params.grad
        proj_result = project_to_poly2(magnet_params.squeeze().detach().cpu().numpy())
        if (proj_result is None):
            magnet_params = torch.tensor(magnet_params).requires_grad_(True)
        else:        
            magnet_params = torch.tensor(proj_result).unsqueeze(0).to(device).requires_grad_(True)






tensor([5.7121e-01, 3.2095e-04, 1.7992e-02, 1.0920e-02, 1.2947e-01, 8.6651e-02],
       device='cuda:0', dtype=torch.float64)


/home/max/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [137]:
magnet_params

tensor([[3.2505, 0.0100, 0.1642, 0.1642, 1.9992, 2.2987]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)

In [98]:
import femmcreator
import magnetdesigner

In [138]:
magnet_params_np = magnet_params.squeeze().detach().cpu().numpy()

print(*list(*(magnet_params.detach().cpu().numpy())))
#['maxCurrentDensity', 'fieldTolerance', 'aper_x', 'aper_y', 'B_design', 'B_real']
dipole = magnetdesigner.designer.get_Dipole(magnet_params_np[4], magnet_params_np[1], 5000, 1e-3, 1e-3, aper_x=magnet_params_np[2], aper_y=magnet_params_np[3], maxCurrentDensity=magnet_params_np[0], B_real=magnet_params_np[5])

femm = femmcreator.FEMM()
femm.Build(dipole.get_femminput)
femm.CreateFEMM('dipole_1')

3.2505109002683796 0.010006523443922517 0.16416221120380337 0.1641613714426864 1.9991632692079533 2.298656318300901


In [94]:
group = dataset_train.dataframe[(dataset_train.dataframe["yoke_x"]==1.4824975) & 
                        (dataset_train.dataframe["yoke_y"]==0.783038) &
                        (dataset_train.dataframe["aper_x"]==0.3) &
                        (dataset_train.dataframe["aper_y"]==0.075)][important_cols_order + ["B0","gfr_x_1e-4","gfr_y_1e-4"]]

In [97]:
network.eval()

def fun(r,i):
    inp = torch.tensor(r[important_cols_order],device=device,dtype=torch.double).unsqueeze(0)
    pred = network(inp).cpu().detach()[0,i].item()
    return pred

group = group.assign(pred_B0=group.apply( lambda r: fun(r,0) ,axis=1))
group = group.assign(pred_gfr_x=group.apply( lambda r: fun(r,1) ,axis=1))
group = group.assign(pred_gfr_y=group.apply( lambda r: fun(r,2) ,axis=1))

group


/tmp/ipykernel_591666/1237417832.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inp = torch.tensor(r[important_cols_order],device=device,dtype=torch.double).unsqueeze(0)
/tmp/ipykernel_591666/1237417832.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inp = torch.tensor(r[important_cols_order],device=device,dtype=torch.double).unsqueeze(0)
/tmp/ipykernel_591666/1237417832.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  i

,maxCurrentDensity,fieldTolerance,aper_x,aper_y,B_design,B_real,B0,gfr_x_1e-4,gfr_y_1e-4,pred_B0,pred_gfr_x,pred_gfr_y
49826,2.0,0.01,0.3,0.075,1.2,0.36,0.358991,0.342158,0.06,0.343749,0.366186,0.076367
49828,2.0,0.01,0.3,0.075,1.2,0.60,0.598375,0.339483,0.06,0.564004,0.341917,0.068916
49832,2.0,0.01,0.3,0.075,1.2,1.08,1.061886,0.298884,0.06,1.004513,0.293379,0.054016
49827,2.0,0.01,0.3,0.075,1.2,0.48,0.478710,0.341072,0.06,0.453877,0.354051,0.072642
49829,2.0,0.01,0.3,0.075,1.2,0.72,0.717982,0.337774,0.06,0.674131,0.329782,0.065191
49835,2.0,0.01,0.3,0.075,1.2,1.44,1.296217,0.169453,0.06,1.334895,0.256976,0.042840
49830,2.0,0.01,0.3,0.075,1.2,0.84,0.836991,0.335543,0.06,0.784259,0.317648,0.061466
49824,2.0,0.01,0.3,0.075,1.2,0.12,0.119470,0.342736,0.06,0.123495,0.390455,0.083817
49834,2.0,0.01,0.3,0.075,1.2,1.32,1.233042,0.204116,0.06,1.224768,0.269110,0.046565
49825,2.0,0.01,0.3,0.075,1.2,0.24,0.239236,0.342575,0.06,0.233622,0.378320,0.080092
